As long as it finishes, 0
The tested force field is:  UFF.json
The tested cif is:  /home/yutao/project/Al-MOF/mil121/RSM0112.cif
The simulation temperature is:  303
This is the final pk values for isotherm workflow:  820016

Go to  /home/yutao/project/Al-MOF/MIL120
Go to  /home/yutao/project/Al-MOF/cau21bpdc
Go to  /home/yutao/project/Al-MOF/WOJJOV
Go to  /home/yutao/project/Al-MOF/MIL-160
Go to  /home/yutao/project/Al-MOF/nott300
Go to  /home/yutao/project/Al-MOF/mof520
Go to  /home/yutao/project/Al-MOF/Al-soc-MOF-1
Go to  /home/yutao/project/Al-MOF/cau10-OCH3
Go to  /home/yutao/project/Al-MOF/cau10H
Go to  /home/yutao/project/Al-MOF/mil121


In [ ]:
'''

This is aiida analysie file which give me the information from binding site workchains and isotherms workchains, 
I wish use them as the same format

'''

'''
This is a plot function, which only receive one kind format and give two way to plot it

1. put all of them in the same figure

'''